In [1]:
import spacy
nlp = spacy.load('en_core_web_md')

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [2]:
import neuralcoref
neuralcoref.add_to_pipe(nlp)

In [2]:
corp = ''
with open('../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-en.txt') as f:
    for line in f:
        corp = corp + line

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-en.txt'

In [3]:
sweCorp = ''
with open('../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-sv.txt') as f:
    for line in f:
        sweCorp = sweCorp + line


FileNotFoundError: [Errno 2] No such file or directory: '../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-sv.txt'

In [5]:
tokenizerEng = spacy.load('en_core_web_md', disable=['tagger', 'parser', 'ner'])
tokenizerSwe = spacy.load('../Models/SwedishModel', disable=['tagger', 'parser', 'ner'])

In [4]:
startTokenEng = 0
startTokenSwe = 0
alignments = {}

with open('../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-en.txt') as engDoc:
    with open('../Data/Datasets/Europarl/Documents/ep-00-01-17/alignments.fw') as fa:
        with open('../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-sv.txt') as sweDoc:
            for lineAli in fa:
                lineEngDoc = engDoc.readline()
                lineSweDoc = sweDoc.readline()
                tokEng = tokenizerEng(lineEngDoc)
                tokSwe = tokenizerSwe(lineSweDoc)

                alignmentListTemp = lineAli.split()
                for ali in alignmentListTemp:
                    aliPair = ali.split('-')
                    key = str(int(aliPair[0]) + startTokenEng)
                    alignments[key] = int(aliPair[1]) + startTokenSwe

                startTokenEng = startTokenEng + len(tokEng)
                startTokenSwe = startTokenSwe + len(tokSwe)


FileNotFoundError: [Errno 2] No such file or directory: '../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-en.txt'

In [ ]:
doc = nlp(corp)


In [7]:
spansList = []

for index, cluster in enumerate(doc._.coref_clusters):
    clusterSpansList = []
    for mention in doc._.coref_clusters[index].mentions:
        mentionSpan = []
        aligns = []
        for i in range(mention.start, mention.end):
            key = str(i)
            if alignments.get(key) is not None:
                aligns.append(alignments.get(key))
        if len(aligns) > 0:
            mentionSpan.append(min(aligns))
            mentionSpan.append(max(aligns) + 1)
        else:
            mentionSpan.append(-1)
        clusterSpansList.append(mentionSpan)
    spansList.append(clusterSpansList)

In [37]:
docSwe = tokenizerSwe(sweCorp)

for ind in range(17774, 17775):
    print(docSwe[ind])


vi


In [13]:
import pickle

with open('../Data/Datasets/Europarl/Documents/ep-00-01-17/corefSpansObject', 'wb') as f:
    pickle.dump(spansList, f)
